<a href="https://colab.research.google.com/github/AndreYang333/LLMdemo/blob/main/Qwen7bRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 安装必要的库
!pip install langchain vllm gptcache modelscope
!pip install transformers==4.32.0 accelerate tiktoken einops scipy transformers_stream_generator==0.0.4 peft deepspeed
!pip install auto-gptq optimum
!pip install --upgrade transformers

In [3]:
from vllm import LLM, SamplingParams
import time
import os
#使用modelscope,如果不设置该环境变量，将会从huggingface下载,中国用
# os.environ['VLLM_USE_MODELSCOPE']='True'

In [4]:
!nvidia-smi


Fri Oct  4 18:22:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
#无量化,最低显存占用约16.5GB
#llm = LLM(model="qwen/Qwen-7B-Chat", trust_remote_code=True)
#int4量化,最低显存占用约7GB
llm = LLM(model="qwen/Qwen-7B-Chat-int4", trust_remote_code=True,gpu_memory_utilization=0.55,max_model_len=1000)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


INFO 10-04 18:22:34 config.py:1652] Downcasting torch.float32 to torch.float16.
WARNING 10-04 18:22:34 config.py:319] gptq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 10-04 18:22:34 llm_engine.py:226] Initializing an LLM engine (v0.6.1.dev238+ge2c6e0a82) with config: model='qwen/Qwen-7B-Chat-int4', speculative_config=None, tokenizer='qwen/Qwen-7B-Chat-int4', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=1000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=gptq, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoin

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


WARNING 10-04 18:22:37 tokenizer.py:156] Using a slow tokenizer. This might cause a significant slowdown. Consider using a fast tokenizer instead.
INFO 10-04 18:22:38 selector.py:217] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
INFO 10-04 18:22:38 selector.py:116] Using XFormers backend.


/usr/local/lib/python3.10/dist-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/usr/local/lib/python3.10/dist-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")


INFO 10-04 18:22:40 model_runner.py:1014] Starting to load model qwen/Qwen-7B-Chat-int4...
INFO 10-04 18:22:41 selector.py:217] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
INFO 10-04 18:22:41 selector.py:116] Using XFormers backend.
INFO 10-04 18:22:42 weight_utils.py:242] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


INFO 10-04 18:23:02 model_runner.py:1025] Loading model weights took 5.5097 GB
INFO 10-04 18:23:05 gpu_executor.py:122] # GPU blocks: 250, # CPU blocks: 512
INFO 10-04 18:23:08 model_runner.py:1329] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 10-04 18:23:08 model_runner.py:1333] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 10-04 18:23:48 model_runner.py:1456] Graph capturing finished in 40 secs.


In [6]:
prompts = [
'''
Andre是不是大帅哥
'''
]

sampling_params = SamplingParams(temperature=0.8,top_k=10, top_p=0.95,max_tokens=256,stop=["<|endoftext|>","<|im_end|>"])
start_time = time.time()
outputs = llm.generate(prompts, sampling_params)
end_time = time.time()
latency = end_time - start_time
print(f"Latency: {latency} seconds")
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt} \nGenerated text: \n{generated_text}")

Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.48s/it, est. speed input: 1.10 toks/s, output: 44.00 toks/s]

Latency: 5.489413738250732 seconds
Prompt: 
Andre是不是大帅哥
 
Generated text: 
这个问题很难回答，因为审美是主观的，不同的人对于“大帅哥”的定义和标准也会有所不同。以下是一些可能的因素，可以帮助我们更好地回答这个问题：

1. 面容特征：Andre的脸部特征包括他的眼睛、鼻子、嘴、下巴等。如果他的这些特征被许多人认为是吸引人的，那么他可能就是大帅哥。

2. 体形：除了脸部特征之外，Andre的体形也是一个重要的因素。如果他的身材健硕，肌肉发达，那么他可能被许多人认为是大帅哥。

3. 着装：Andre的穿着打扮也会影响人们对他的看法。如果他的穿着时尚，符合他的年龄、身材和个性，那么他可能被许多人认为是大帅哥。

4. 表情和态度：Andre的表情和态度也会影响人们对他的看法。如果他自信、友好、有礼貌，那么他可能被许多人认为是大帅哥。

5. 社交网络影响力：如果Andre在社交媒体上有大量的粉丝和关注者，那么他可能被许多人认为是大帅哥。

因此，要回答这个问题，需要考虑上述的所有因素，并结合不同人的看法和评价。


In [ ]:
# 使用外部知识库
!pip install faiss-gpu
!pip install -U langchain-community
!pip install sentence-transformers

In [8]:
from langchain.vectorstores import FAISS

from langchain.embeddings import HuggingFaceEmbeddings

# 指定 Hugging Face 的模型名称
model_name = "sentence-transformers/all-MiniLM-L6-v2"

# 创建 Hugging Face 嵌入对象
embeddings = HuggingFaceEmbeddings(model_name=model_name)




<ipython-input-8-a922630eae15>:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
knowledges=["Andre是大帅哥"]
vectorstore = FAISS.from_texts(
    knowledges, embedding=embeddings
)
retriever = vectorstore.as_retriever()

In [10]:
retriever.invoke("Andre")[0].page_content

'Andre是大帅哥'

In [52]:
system_prompt = ''
sampling_params = SamplingParams(temperature=0.8,top_k=10, top_p=0.95,max_tokens=256,stop=["<|endoftext|>","<|im_end|>"])
def generateText(prompts):
    prompt = prompts.pop()
    print("prompt:",prompt)
    background = retriever.invoke(prompt)[0].page_content
    print("background:",background)

    input = str(f'''\
    {system_prompt+"背景知识:"+background+"。用户的问题是:" + prompt}''')

    print(input)
    output = llm.generate(input, sampling_params)
    generated_text = output[0].outputs[0].text
    generated_text=generated_text.replace("<|im_start|>","").replace("<|im_end|>","")
    print(generated_text)
    return output
    # ret = {"text": str(generated_text)}
